In [1]:
from grounding_concepts import *

In [2]:
import re
import string
import pandas as pd

In [3]:
from __future__ import absolute_import

import argparse
import csv
import logging
import os
import random
import sys
from io import open

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange
import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random

In [4]:
#file_path = "../datasets/Train_Final_complete.csv"

In [5]:
file_path = './../openbook/train_openbook.csv'

In [6]:
f = './../datasets/arc_data/'

In [7]:
data = pd.read_csv(file_path)

In [8]:
data

,Unnamed: 0,answerkey,hypA,hypB,hypC,hypD,preA,preB,preC,preD
0,0,D,The sun is responsible for puppies learning ne...,The sun is responsible for children growing up...,The sun is responsible for flowers wilting in ...,"The sun is responsible for plants sprouting, b...",koto was always learning new tricks.\nleica is...,experts are now starting to tell us that child...,ray flowers are wilting.\nsubject: a vase with...,show to the class plants on a dry potted soil ...
1,1,D,When standing miles away from Mount Rushmore t...,When standing miles away from Mount Rushmore t...,When standing miles away from Mount Rushmore t...,When standing miles away from Mount Rushmore t...,the location of crazy horse monument is about ...,the location of crazy horse monument is about ...,the location of crazy horse monument is about ...,the location of crazy horse monument is about ...
2,2,C,When food is reduced in the stomach the mind n...,When food is reduced in the stomach take a sec...,When food is reduced in the stomach nutrients ...,When food is reduced in the stomach reader's d...,the stomach of the calf needs time to develop ...,starfish will spit out their stomach and wrap ...,digestion-digestion of food and absorption of ...,one of her works was published on the cover of...
3,3,C,Stars are warm lights that float,Stars are made out of nitrate,Stars are great balls of gas burning billions ...,Stars are lights in the sky,some containers float in warm water.\nsome pla...,it is made by reacting sodium carbonate with s...,there are stars that are billions of times far...,"there, in private, the stars descended, tatted..."
4,4,D,You can make a telescope with a straw,You can make a telescope with a Glass,You can make a telescope with a Candle,You can make a telescope with a mailing tube,bread wheat straw is used for thatching and we...,afield-glass; a telescope.\nla2o2 is used to m...,make a candle\ncandle holders also make great ...,all prints are shipped in a sturdy mailing tub...
...,...,...,...,...,...,...,...,...,...,...
4952,4952,A,A bulldozer alters the area of skyscrapers,A bulldozer alters the area of the stock market,A bulldozer alters the area of air,A bulldozer alters the area of water,the bocagrande ( big mouth ) is an area known ...,a stock index or stock market index is a measu...,most air resistance alters motion.\nwater vapo...,it alters to siderotil by loss of water .\nthi...
4953,4953,A,An organism that can survive without the help ...,An organism that can survive without the help ...,An organism that can survive without the help ...,An organism that can survive without the help ...,a single-cell organism can survive without the...,a single-cell organism can survive without the...,a single-cell organism can survive without the...,a single-cell organism can survive without the...
4954,4954,B,The nimbleness of this animal is a key adaptio...,The nimbleness of this animal is a key adaptio...,The nimbleness of this animal is a key adaptio...,The nimbleness of this animal is a key adaptio...,praying mantis predators\npraying mantis - one...,"this physiology is beneficial, because it dist...","this physiology is beneficial, because it dist...","this physiology is beneficial, because it dist..."
4955,4955,A,Birds will have different kinds of beaks depen...,Birds will have different kinds of beaks depen...,Birds will have different kinds of beaks depen...,Birds will have different kinds of beaks depen...,birds' different-shaped beaks\nbirds different...,birds' different-shaped beaks\nbirds different...,birds' different-shaped beaks\nbirds different...,birds' different-shaped beaks\nbirds different...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4957 entries, 0 to 4956
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4957 non-null   int64 
 1   answerkey   4957 non-null   object
 2   hypA        4957 non-null   object
 3   hypB        4957 non-null   object
 4   hypC        4957 non-null   object
 5   hypD        4957 non-null   object
 6   preA        4957 non-null   object
 7   preB        4957 non-null   object
 8   preC        4957 non-null   object
 9   preD        4957 non-null   object
dtypes: int64(1), object(9)
memory usage: 387.4+ KB


In [10]:
class openbookExample(object):
    """A single training/test example for the ARC dataset."""

    def __init__(self,
                 ob_id,
                 context_sentences,
                 hypothesis,
                 question,
                 ending_0,
                 ending_1,
                 ending_2,
                 ending_3,
                 label=None):
        self.ob_id = ob_id
        self.context_sentences = context_sentences
        self.hypothesis = hypothesis
        self.question = question
        self.endings = [
            ending_0,
            ending_1,
            ending_2,
            ending_3,
        ]
        self.label = label

In [11]:
class InputFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'gpre' : gpre,
                'ghyp' : ghyp,
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, gpre, ghyp, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label


In [12]:
def read_ob_examples(input_file, is_training=True):
    '''
    :returns :
    training dataset.
    list of a single training/testing example for the ARC dataset.
    A single ARC example is a object with following format :
    (context sentence, hypothesis, option a, option b , ... , label)

    '''
    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        lines = []
        for line in reader:
            if sys.version_info[0] == 2:
                line = list(unicode(cell, 'utf-8') for cell in line)
            lines.append(line)

    if is_training and lines[0][1] != 'answerkey':
        raise ValueError(
            "For training, the input file must contain a label column."
        )

    examples = []
    for line in lines[1:]:
        context_sentences = [line[6], line[7], line[8],line[9]]
        hypothesis = [line[2], line[3], line[4], line[5]]
        label = ord(line[1]) - ord('A') if is_training else None

        '''examples.append(
                                    ArcExample(arc_id=line[-2], context_sentences=context_sentences, hypothesis = hypothesis, question=line[-1], ending_0=line[1],
                                               ending_1=line[2], ending_2=line[3], ending_3=line[4], label=label))
                        '''
        examples.append(
                                    openbookExample(ob_id=line[0], context_sentences=context_sentences, hypothesis = hypothesis, question=line[-3], ending_0=line[1],
                                               ending_1=line[2], ending_2=line[3], ending_3=line[4], label=label))
    return examples

In [13]:
examples = read_ob_examples(file_path)

In [14]:
len(examples)

4957

In [15]:
examples[0].context_sentences[0]

'koto was always learning new tricks.\nleica is adept at learning new tricks and loves people.\nunlike old dogs, old ferrets love learning new tricks.\nthey respond well to parrot training techniques and are willing students for learning new tricks.\na cockatoo is capable of learning new tricks and skills until the end of his life.\ncockatiels are more adept at learning beak tricks than claw tricks.\nthe artisans who are bringing new telescope mirrors into the world are learning new tricks nearly every day.\n[13] sun conures are capable of learning a great deal of tricks and can even perform in front of a live audience.\nlearning a few tricks on finding an object\'s volume.\nvery alert and learning tricks so quickly!\nbut, a responsible breeder will guarantee their puppies for life.\nresponsible breeders make sure their puppies\' new families know they can turn to them with any questions or problems that arise throughout the puppies\' lives.\nand new tricks .\nthere are no special tric

In [16]:
d = examples[0].context_sentences[0]

In [17]:
d = re.sub('\n', '', d)

In [18]:
d

'koto was always learning new tricks.leica is adept at learning new tricks and loves people.unlike old dogs, old ferrets love learning new tricks.they respond well to parrot training techniques and are willing students for learning new tricks.a cockatoo is capable of learning new tricks and skills until the end of his life.cockatiels are more adept at learning beak tricks than claw tricks.the artisans who are bringing new telescope mirrors into the world are learning new tricks nearly every day.[13] sun conures are capable of learning a great deal of tricks and can even perform in front of a live audience.learning a few tricks on finding an object\'s volume.very alert and learning tricks so quickly!but, a responsible breeder will guarantee their puppies for life.responsible breeders make sure their puppies\' new families know they can turn to them with any questions or problems that arise throughout the puppies\' lives.and new tricks .there are no special tricks to learning how to grow

In [19]:
len(d)

3731

In [20]:
info = (d[:512] + '..') if len(d) > 512 else d

In [21]:
info

'koto was always learning new tricks.leica is adept at learning new tricks and loves people.unlike old dogs, old ferrets love learning new tricks.they respond well to parrot training techniques and are willing students for learning new tricks.a cockatoo is capable of learning new tricks and skills until the end of his life.cockatiels are more adept at learning beak tricks than claw tricks.the artisans who are bringing new telescope mirrors into the world are learning new tricks nearly every day.[13] sun conu..'

In [22]:
def proces_ob(data):
    nlp = spacy.load('en_core_web_sm',disable=['ner','parser','textcat'])
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    output_path = "ob_concepts"+".%d.mcp"
    concepts = []
    for i,example in tqdm(enumerate(data)):
        x = []
        for j in tqdm(range(4)):
            sents = example.context_sentences[j]
            sent = re.sub('\n', '', sents)[:1024]
            #sent = (d[:512] + '..') if len(d) > 512 else d
            res = match_mentioned_concepts_context(nlp,context_sents=sent,
                                                   hypo_sents=example.hypothesis[j],
                                                   answer=example.endings[j])
            x.append(res)
        concepts.append(x)
        
    return concepts
    #with open(output_path, 'w') as fo:
        #json.dump(concepts, fo)
        

In [23]:
examples[0]

In [24]:
concepts = proces_ob(examples[4551:4553])

In [25]:
len(concepts[0][0][0]['premise'])

37

In [26]:
concepts[0]

[[{'sent': ' ',
   'option': 'A',
   'premise': ['whethere',
    'heats',
    'solid',
    'change',
    'phase',
    'refer_to',
    'difference',
    'change_into',
    'freezed',
    'melt',
    'above_us',
    'liquid_state',
    'turn_to',
    'whether',
    'liquid_or',
    'heat',
    'froze',
    'point',
    'mattering',
    'state',
    'liquids',
    'melted',
    'imply',
    'incidentally',
    'freeze',
    'turn',
    'point_of',
    'eventually',
    'substance',
    'matter',
    'phase_change',
    'and_turn',
    'orientation',
    'refer',
    'transform',
    'liquid',
    'transforming'],
   'hypothesis': ['heats',
    'solid',
    'freezed',
    'melt',
    'above_us',
    'cool',
    'heat',
    'froze',
    'point',
    'cools',
    'accurate',
    'liquids',
    'liquid',
    'melted',
    'being_cool',
    'freeze'],
   'ans': ['a']}],
 [{'sent': ' ',
   'option': 'Which is most accurate? melting is when solids are heated above their melting point; freezing i

In [27]:
def process(filename, batch_id=-1):


    nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'textcat'])
    nlp.add_pipe(nlp.create_pipe('sentencizer'))

    sents = []
    answers = []
    with open(filename, 'r') as f:
        lines = f.read().split("\n")


    for line in tqdm(lines, desc="loading file"):
        if line == "":
            continue
        j = json.loads(line)
        for statement in j["statements"]:
            sents.append(statement["statement"])
        for answer in j["question"]["choices"]:
            answers.append(answer["text"])


    if batch_id >= 0:
        output_path = filename + ".%d.mcp" % batch_id
        batch_sents = list(np.array_split(sents, 100)[batch_id])
        batch_answers = list(np.array_split(answers, 100)[batch_id])
    else:
        output_path = filename + ".mcp"
        batch_sents = sents
        batch_answers = answers

    res = match_mentioned_concepts(nlp, sents=batch_sents, answers=batch_answers, batch_id=batch_id)
    return res
    #with open(output_path, 'w') as fo:
        #json.dump(res, fo)

In [28]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'textcat'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [29]:
sents = []
answers = []
with open("../datasets/csqa_new/dev_rand_split.jsonl.statements", 'r') as f:
    lines = f.read().split("\n")

In [30]:
lines[0]

'{"answerKey": "A", "id": "1afa02df02c908a558b4036e80242fac", "question": {"question_concept": "revolving door", "choices": [{"label": "A", "text": "bank"}, {"label": "B", "text": "library"}, {"label": "C", "text": "department store"}, {"label": "D", "text": "mall"}, {"label": "E", "text": "new york"}], "stem": "A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?"}, "statements": [{"label": true, "statement": "A revolving door is convenient for two direction travel, but it also serves as a security measure at a bank."}, {"label": false, "statement": "A revolving door is convenient for two direction travel, but it also serves as a security measure at a library."}, {"label": false, "statement": "A revolving door is convenient for two direction travel, but it also serves as a security measure at a department store."}, {"label": false, "statement": "A revolving door is convenient for two direction travel, but it also serves as a secu

In [31]:
len(lines)

1222

In [32]:
l = lines[:10]

In [33]:
len(l)

10

In [34]:
for line in tqdm(l, desc="loading file"):
    if line == "":
        continue
    j = json.loads(line)
    for statement in j["statements"]:
        sents.append(statement["statement"])
    for answer in j["question"]["choices"]:
        answers.append(answer["text"])

In [35]:
sents[0]

'A revolving door is convenient for two direction travel, but it also serves as a security measure at a bank.'

In [36]:
answers[0]

'bank'

In [37]:
with open("../datasets/csqa_new/arc_challenge.json", 'r') as f:
    arc_lines = f.read().split("\n")

In [38]:
len(arc_lines)

15539

In [39]:
arc_lines[0:10]

['{',
 ' "ARC-Challenge-Train.jsonl-Mercury_SC_415702": [',
 '  "(in this activity, let the pupils understand that in rubbing their hands together, heat is produced and the heat produced makes their hands warm.)\\nwhen the skin surface is exposed to heat, warm blood flushes into the ______________ and heat radiates from the skin surface.\\nglabrous skin covers the surface of the palms of the hands, soles of the feet, and flexor surfaces of the digits.\\nmajor heat loss is evaporation of h2o on skin of infant - put in dry, warm blankets.\\nskin: very dry, fair, warm\\nsevere rash with skin swelling (including on the palms of the hands and soles of the feet)\\nwhen the body becomes too warm, blood vessels in the skin dilate, allowing heat to escape through the surface of the skin.\\nalopecia; dry/itchy skin w/ desquamation of palms and soles\\nridges warm up and dry out quickly.\\nthe mechanical energy of rubbing your hands together is converted to thermal (heat) energy.\\nthis is becaus

In [40]:
rem_data = []
rem_data.append(examples[250])
rem_data.append(examples[500])

In [41]:
match_ = proces_arc(rem_data)

NameError: name 'proces_arc' is not defined

In [ ]:
len(match_)

In [ ]:
match_[0][0]

In [ ]:
with open("arc_rem.mcp","w") as f:
    json.dump(match_,f)